In [2]:
import looker_sdk
import pandas as pd
import numpy as np
import configparser
import json
import os
import matplotlib.pyplot as plt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from textwrap import wrap
from datetime import date
import plotly.express as px

In [3]:
# Base URL for API. Do not include /api/* in the url
base_url = "https://looker.clarityhs.com:19999"

#Access login info from .ini
config = configparser.ConfigParser()
config.read('/Users/cristiandeleon/RTFH_DataDashboard/env/.ini')

# API 3 client id
client_id = str(config['login']['client_id'])# Enter your client_id here
# API 3 client secret
client_secret = str(config['login']['client_secret']) # Enter your client_secret here

# Set to false if testing locally against self-signed certs. Otherwise leave True
verify_ssl=True

os.environ["LOOKERSDK_BASE_URL"] = base_url #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "3.1" #3.1 is the default version. You can change this to 4.0 if you want.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  client_id #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = client_secret #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.

print("All environment variables set.")

sdk = looker_sdk.init31()
print('Looker SDK 3.1 initialized successfully.')

my_user = sdk.me()

#Output is an instance of the User model, but can also be read like a python dict. This applies to all Looker API calls that return Models.
#Example: The following commands return identical output. Feel free to use whichever style is more comfortable for you.

print(my_user.first_name) #Model dot notation
print(my_user["first_name"]) #Dictionary

All environment variables set.
Looker SDK 3.1 initialized successfully.
Cristian
Cristian


# Pull Look Data for Newsletter Dashboard

In [5]:
# Pull look data
temp1 = sdk.run_look(look_id=79680, result_format='json', limit=500000)

In [6]:
# Transform look data into pandas dataframe
n_temp = json.loads(temp1)
df = pd.DataFrame.from_dict(n_temp)
df
# Turn date columns into type datetime
df['enrollments.start_date'] = pd.to_datetime(df['enrollments.start_date'], errors='coerce')
df['enrollments.end_date'] = pd.to_datetime(df['enrollments.end_date'])
df['household_move_in_date.move_in_date'] = pd.to_datetime(df['household_move_in_date.move_in_date'])

# Convert age column as type int and identify null values using -1
df['clients.age'] = df['clients.age'].fillna(-1).astype(int)
df = df[~pd.isnull(df['enrollments.start_date'])].reset_index(drop=True)

In [8]:
temp4 = sdk.run_look(look_id=79431, result_format='json', limit=200000)
n_temp4 = json.loads(temp4)
df3 = pd.DataFrame.from_dict(n_temp4)
df3['referrals.date_date'] = pd.to_datetime(df3['referrals.date_date'])

In [18]:
PATH = f'assets/{date.today()}'
if not os.path.isdir(PATH):
    os.mkdir(PATH)

# Functions utilized in this notebook

In [7]:
"""

### Inflow Functions

"""
def getActiveClientsDF(df, rep_period_start, rep_period_end):
    '''
    Filter dataframe for only those clients that were active in the specified reporting period.
    '''
    return df[(df['enrollments.start_date']<rep_period_end) & ((df['enrollments.end_date']>=rep_period_start) | pd.isnull(df['enrollments.end_date']))].reset_index(drop=True).copy()

def getActiveClientsCount(df, rep_period_start, rep_period_end):
    '''
    Use get the count of Unique ID's from the dataframe returned by the getActiveClientsDF() function call.
    '''
    return getActiveClientsDF(df, rep_period_start, rep_period_end)['clients.unique_identifier'].nunique()#df[(df['enrollments.start_date']<rep_period_end) & ((df['enrollments.end_date']>=rep_period_start) | pd.isnull(df['enrollments.end_date']))]['clients.unique_identifier'].nunique()

def getClientsEnrolledInRepPeriodDF(df, rep_period_start, rep_period_end):
    '''
    Returns a dataframe filtered for only those clients that have an Enrollment Start Date in the reporting period.
    '''
    return df[(df['enrollments.start_date']<rep_period_end) & (df['enrollments.start_date']>=rep_period_start)].reset_index(drop=True).copy()

def getClientsEnrolledCount(df, rep_period_start, rep_period_end):
    '''
    Returns the number of Unique ID's in the dataframe returned by the getClientsEnrolledInRepPeriodDF() function call. 
    '''
    return getClientsEnrolledInRepPeriodDF(df, rep_period_start, rep_period_end)['clients.unique_identifier'].nunique()

def getClientsEnrolledSet(df, rep_period_start, rep_period_end):
    '''
    Returns the set of Unique ID's in the dataframe returned by the getClientsEnrolledInRepPeriodDF() function call. 
    '''
    return set(getClientsEnrolledInRepPeriodDF(df, rep_period_start, rep_period_end)['clients.unique_identifier'].unique())

def getClientsActiveForEntireRepPeriodSet(df, rep_period_start, rep_period_end):
    '''
    Returns the set of Unique ID's for clients that were served every day in the reporting period.
    '''
    return set(df[(df['enrollments.start_date']<rep_period_start) & ((df['enrollments.end_date']>=rep_period_end) | pd.isnull(df['enrollments.end_date']))]['clients.unique_identifier'].unique())

def getFirstTimeHomelessDF(df, rep_period_start, rep_period_end):
    '''
    Returns a dataframe of each clients first enrollment by start date and then filters for 
    clients who's first enrollment was in the reporting period.
    '''
    idx = df.groupby('clients.unique_identifier')['enrollments.start_date'].idxmin()
    df = df.loc[idx].reset_index(drop=True)
    return df[(df['enrollments.start_date']<rep_period_end) & (df['enrollments.start_date']>=rep_period_start)].reset_index(drop=True).copy()

def getFirstTimeHomelessCount(df, rep_period_start, rep_period_end):
    '''
    Returns the number of unique ID's returned by the getFirstTimeHomelessDF() function call.
    '''
    return getFirstTimeHomelessDF(df, rep_period_start, rep_period_end)['clients.unique_identifier'].nunique()
"""
### Outflow Functions

"""
def getClientsExitedInReportingPeriodDF(df, rep_period_start, rep_period_end):
     return df[(df['enrollments.end_date']>=rep_period_start) & (df['enrollments.end_date']<rep_period_end) & ~pd.isnull(df['enrollments.end_date'])].reset_index(drop=True).copy()

def getClientsExitedInReportingPeriodSet(df, rep_period_start, rep_period_end):
    return set(getClientsExitedInReportingPeriodDF(df, rep_period_start, rep_period_end)['clients.unique_identifier'].unique())

def getClientsExitedAndHoused(df, rep_period_start, rep_period_end):
    temp_df = getClientsExitedInReportingPeriodDF(df, rep_period_start, rep_period_end)
    return temp_df[temp_df['last_screen.housed_on_exit']=='Housed'].reset_index(drop=True)

def getExitToMoveInDF(df, rep_period_start, rep_period_end):
    exit_to_or_move_in_enroll_set = set(
                                        list(getClientsExitedAndHoused(df, rep_period_start, rep_period_end)['enrollments.id'].unique()) + 
                                        list(getClientsWithMoveInDF(df, rep_period_start, rep_period_end)['enrollments.id'].unique())
                                    )
    return df[df['enrollments.id'].isin(exit_to_or_move_in_enroll_set)].reset_index(drop=True)

""" 

### Housing Move-In Functions

"""

def getClientsWithMoveInDF(df, rep_period_start, rep_period_end):
    return df[~pd.isnull(df['household_move_in_date.move_in_date']) & (df['household_move_in_date.move_in_date']>=rep_period_start) & (df['household_move_in_date.move_in_date']<rep_period_end)].reset_index(drop=True)

def getSetOfClientsWithMoveIn(df, rep_period_start, rep_period_end):
    return set(getClientsWithMoveInDF(df, rep_period_start, rep_period_end)['clients.unique_identifier'].unique())

def getCountOfClientsWithMoveIn(df, rep_period_start, rep_period_end):
    return len(getSetOfClientsWithMoveIn(df, rep_period_start, rep_period_end))


"""

### Subpopulation Count Functions

"""
def getSeniorsServedDF(df):
    return df[df['clients.age']>=55].reset_index(drop=True)

def getSeniorsServedCount(df):
    return df[df['clients.age']>=55]['clients.unique_identifier'].nunique()

def getTAYServedCount(df):
    return df[(df['clients.age']>=18) & (df['clients.age']<=24)]['clients.unique_identifier'].nunique()

def getTAYServedDF(df):
    return df[(df['clients.age']>=18) & (df['clients.age']<=24)].reset_index(drop=True)

def getVeteransServedCount(df):
    return df[df['static_demographics.veteran_text']=="Yes"]['clients.unique_identifier'].nunique()

def getVeteransServedDF(df):
    return df[df['static_demographics.veteran_text']=="Yes"].reset_index(drop=True)

def getFamiliesServedCount(df):
    return df[(df['household_makeup.household_type']=="Household with Children") & (df['entry_screen.head_of_household']=='Yes')]['clients.unique_identifier'].nunique()

def getFamiliesServedDF(df):
    return df[(df['household_makeup.household_type']=="Household with Children") & (df['entry_screen.head_of_household']=='Yes')].reset_index(drop=True)

# Race Data Pull for New To System, Active and Housed Sections

In [9]:
temp = df.copy()

monthly_time_periods_d = {
    "Monthly": 1,
    "Quarterly": 4,
    "6 - Month": 6,
    "Annual": 12
} 

list_of_dfs_2 = []

REPORT_START_DATE = pd.to_datetime('2022-09-01')
while REPORT_START_DATE<pd.to_datetime(date.today().strftime("%Y-%m-01")):
    for time_range in monthly_time_periods_d:
        if time_range=="Monthly":
            
            # Set repording period start and end
            rep_period_start = pd.to_datetime(REPORT_START_DATE)
            rep_period_end = rep_period_start + relativedelta(months=1)

            # Set a lookback period
            lookback_start = rep_period_start - relativedelta(months=1)
            lookback_end = rep_period_start
    
        else:
            rep_period_start = pd.to_datetime(REPORT_START_DATE) - relativedelta(months=monthly_time_periods_d[time_range]-1)
            rep_period_end = rep_period_start + relativedelta(months=monthly_time_periods_d[time_range])

            # Set a lookback period
            lookback_start = rep_period_start - relativedelta(months=monthly_time_periods_d[time_range])
            lookback_end = rep_period_start
        
        # Section 1 New to System Metrics by Race
        section1_df = pd.concat([
            pd.DataFrame(getFirstTimeHomelessDF(temp, rep_period_start, rep_period_end).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "FTH Count by Race"}).T, 
            pd.DataFrame(getClientsExitedAndHoused(temp, rep_period_start, rep_period_end).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Housed Count by Race"}).T, 
            pd.DataFrame(getClientsEnrolledInRepPeriodDF(temp, rep_period_start, rep_period_end).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "New Program Entries Count by Race"}).T,
            pd.DataFrame(df3[(df3['referrals.date_date']>=rep_period_start) & (df3['referrals.date_date']<rep_period_end)].groupby(['static_demographics.race_text'])['client.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"client.unique_identifier": "New Referrals Count by Race"}).T,
            ]).T

        # Section 2 Active Metrics by Race
        section2_df = pd.concat([
            pd.DataFrame(getActiveClientsDF(temp, rep_period_start, rep_period_end).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Active Count by Race"}).T, 
            pd.DataFrame(getSeniorsServedDF(getActiveClientsDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Active Seniors Count by Race"}).T, 
            pd.DataFrame(getTAYServedDF(getActiveClientsDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Active TAY Count by Race"}).T, 
            pd.DataFrame(getVeteransServedDF(getActiveClientsDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Active Veterans Count by Race"}).T, 
            pd.DataFrame(getFamiliesServedDF(getActiveClientsDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Active Families Count by Race"}).T
            ]).T
        

        # Section 3 Housed Metrics by Race

        section3_df = pd.concat([
            pd.DataFrame(getSeniorsServedDF(getExitToMoveInDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Housed Seniors Count by Race"}).T, 
            pd.DataFrame(getTAYServedDF(getExitToMoveInDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Housed TAY Count by Race"}).T,
            pd.DataFrame(getVeteransServedDF(getExitToMoveInDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Housed Veterans Count by Race"}).T,
            pd.DataFrame(getFamiliesServedDF(getExitToMoveInDF(temp, rep_period_start, rep_period_end)).groupby(['static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Housed Families Count by Race"}).T
            ]).T

        data_merged_by_race_df = pd.merge(pd.merge(section1_df, section2_df, on='static_demographics.race_text', how='outer'), section3_df, on='static_demographics.race_text', how='outer').T
        data_merged_by_race_df = data_merged_by_race_df.T.reset_index()

        for col in data_merged_by_race_df.columns:
            if 'by Race' in col:
                data_merged_by_race_df[f'{col} by Percent'] = data_merged_by_race_df[col]/data_merged_by_race_df[col].sum()
        data_merged_by_race_df['Reporting Month'] = REPORT_START_DATE.strftime("%b-%Y")
        data_merged_by_race_df['Reporting Window'] = time_range

        list_of_dfs_2.append(data_merged_by_race_df)

    REPORT_START_DATE = REPORT_START_DATE + relativedelta(months=1)

In [19]:
df2 = pd.concat(list_of_dfs_2)
df2[[x for x in df2.columns if 'Race' in x]] = df2[[x for x in df2.columns if 'by Race' in x]].fillna(0).astype(int)
df2.to_csv(f'{PATH}/newsletter_counts_by_race.csv')

# Data Pull for the Active Donut Chart

In [10]:
abbrev_dict = {
    'Safe Haven':'SH', 
    'Other':'OTH', 
    'Transitional Housing':'TH',
    'Homelessness Prevention':'HP', 
    'Street Outreach':'OUT', 
    'Coordinated Entry':'CE',
    'Day Shelter':'DS', 
    'Emergency Shelter':'ES', 
    'Services Only':'SO',
    'Permanent Housing':'PH'
}

def proj_type_rename(row):
    if row['programs.project_type_code'] in ['PH - Housing Only', 'PH - Housing with Services (no disability required for entry)', 'PH - Permanent Supportive Housing (disability required for entry)','PH - Rapid Re-Housing']:
        return 'Permanent Housing'
    else:
        return row['programs.project_type_code']


def append_abbrev(row):
    return f"{row['Project Type']} ({abbrev_dict[row['Project Type']]})"


temp = df.copy()

monthly_time_periods_d = {
    "Monthly": 1,
    "Quarterly": 4,
    "6 - Month": 6,
    "Annual": 12
} 

list_of_dfs_3 = []

REPORT_START_DATE = pd.to_datetime('2022-09-01')
while REPORT_START_DATE<pd.to_datetime(date.today().strftime("%Y-%m-01")):
    for time_range in monthly_time_periods_d:
        if time_range=="Monthly":
            
            # Set repording period start and end
            rep_period_start = pd.to_datetime(REPORT_START_DATE)
            rep_period_end = rep_period_start + relativedelta(months=1)

            # Set a lookback period
            lookback_start = rep_period_start - relativedelta(months=1)
            lookback_end = rep_period_start
    
        else:
            rep_period_start = pd.to_datetime(REPORT_START_DATE) - relativedelta(months=monthly_time_periods_d[time_range]-1)
            rep_period_end = rep_period_start + relativedelta(months=monthly_time_periods_d[time_range])

            # Set a lookback period
            lookback_start = rep_period_start - relativedelta(months=monthly_time_periods_d[time_range])
            lookback_end = rep_period_start
            
        # Filter df for active clients
        active_clients_by_proj_type_df = getActiveClientsDF(temp, rep_period_start, rep_period_end)
        # Change the Project Type to group PH programs
        active_clients_by_proj_type_df['Project Type'] = active_clients_by_proj_type_df.apply(lambda row: proj_type_rename(row), axis=1)
        # Summing the client count by their new Project Type grouping
        active_clients_by_proj_type_df = active_clients_by_proj_type_df.groupby(['Project Type', 'static_demographics.race_text'])['clients.unique_identifier'].nunique().sort_values()
        # Convert groupby object into dataframe
        active_clients_by_proj_type_df = pd.DataFrame(active_clients_by_proj_type_df).reset_index()
        # Reduce active_clients_by_proj_type_df dataframe to 2 columns
        active_clients_by_proj_type_df = active_clients_by_proj_type_df[['Project Type', 'static_demographics.race_text', 'clients.unique_identifier']].copy()
        active_clients_by_proj_type_df = pd.merge(active_clients_by_proj_type_df.reset_index(),pd.DataFrame(pd.Series(abbrev_dict)).reset_index().rename(columns={0:'Abbrev', 'index':'Project Type'}), on='Project Type')
        active_clients_by_proj_type_df['Project Type'] = active_clients_by_proj_type_df.apply(lambda row: append_abbrev(row), axis=1)
        active_clients_by_proj_type_df = active_clients_by_proj_type_df.set_index('Project Type').drop('index', axis=1)
        active_clients_by_proj_type_df = active_clients_by_proj_type_df.reset_index()
        active_clients_by_proj_type_df['Active Clients Count Percent'] = active_clients_by_proj_type_df['clients.unique_identifier']/active_clients_by_proj_type_df['clients.unique_identifier'].sum()

        active_clients_by_proj_type_df['Reporting Month'] = REPORT_START_DATE.strftime("%b-%Y")
        active_clients_by_proj_type_df['Reporting Window'] = time_range
        
        list_of_dfs_3.append(active_clients_by_proj_type_df)
    REPORT_START_DATE = REPORT_START_DATE + relativedelta(months=1)

In [20]:
df3 = pd.concat(list_of_dfs_3).rename(columns={'clients.unique_identifier':'Active Clients Count'})
df3.to_csv(f'{PATH}/newsletter_active_counts_by_proj_type_by_race.csv')

# Data for Housed Situations Bar Chart

In [11]:
permanent_sit = [
        'Staying or living with family, permanent tenure', 
        'Staying or living with friends, permanent tenure', 
        'Rental by client, no ongoing housing subsidy',
        'Rental by client, with VASH housing subsidy',
        'Rental by client, with other ongoing housing subsidy', 
        'Permanent housing (other than RRH) for formerly homeless persons'
        ]

# A function to identify whether an enrollment had an exit or a move-in
def exit_or_move_in(row):
    if not pd.isnull(row['household_move_in_date.move_in_date']) and row['household_move_in_date.move_in_date']>=rep_period_start and row['household_move_in_date.move_in_date']<=rep_period_end:
        return "Move In to PH"
    else:
        if not pd.isnull(row['enrollments.end_date']) and row['enrollments.end_date']>=rep_period_start and row['enrollments.end_date']<=rep_period_end:
            return f"{row['last_screen.exit_destination_text']}"

# A function to group the positive exit destination categories
def ph_category1(row):
    moved_in_with_family = [
        'Staying or living with family, permanent tenure', 
        'Staying or living with friends, permanent tenure'
        ]
    rental_by_client = [
        'Rental by client, no ongoing housing subsidy',
        'Rental by client, with VASH housing subsidy',
        'Rental by client, with other ongoing housing subsidy',
        'Rental by client in a public housing unit',
        'Rental by client, with HCV voucher (tenant or project based)',
        'Rental by client, with RRH or equivalent subsidy',
        'Rental by client, with GPD TIP housing subsidy'
        ]
    permanent_housing = [
        'Permanent housing (other than RRH) for formerly homeless persons', 
        'Moved from one HOPWA funded project to HOPWA PH'
        ]
    owned_by_client = [
        'Owned by client, no ongoing housing subsidy',
        'Owned by client, with ongoing housing subsidy'
    ]

    if row['exit_dest_or_move_in']=='Move In to PH':
        return 'Move In to PH'
    elif row['exit_dest_or_move_in'] in moved_in_with_family:
        return 'Moved in with Family'
    elif row['exit_dest_or_move_in'] in rental_by_client:
        return 'Rental by Client'
    elif row['exit_dest_or_move_in'] in permanent_housing:
        return 'Other Permanent Housing'
    elif row['exit_dest_or_move_in'] in owned_by_client:
        return 'Owned by Client'
    else:
        return f"CHECK -- {row['exit_dest_or_move_in']}"

temp = df.copy()

monthly_time_periods_d = {
    "Monthly": 1,
    "Quarterly": 4,
    "6 - Month": 6,
    "Annual": 12
} 

list_of_dfs_4 = []

REPORT_START_DATE = pd.to_datetime('2022-09-01')
while REPORT_START_DATE<pd.to_datetime(date.today().strftime("%Y-%m-01")):
    for time_range in monthly_time_periods_d:
        if time_range=="Monthly":
            
            # Set repording period start and end
            rep_period_start = pd.to_datetime(REPORT_START_DATE)
            rep_period_end = rep_period_start + relativedelta(months=1)

            # Set a lookback period
            lookback_start = rep_period_start - relativedelta(months=1)
            lookback_end = rep_period_start
    
        else:
            rep_period_start = pd.to_datetime(REPORT_START_DATE) - relativedelta(months=monthly_time_periods_d[time_range]-1)
            rep_period_end = rep_period_start + relativedelta(months=monthly_time_periods_d[time_range])

            # Set a lookback period
            lookback_start = rep_period_start - relativedelta(months=monthly_time_periods_d[time_range])
            lookback_end = rep_period_start

        total_exit_to_or_move_in_df = getExitToMoveInDF(temp, rep_period_start, rep_period_end)
        total_exit_to_or_move_in_df['exit_dest_or_move_in'] = total_exit_to_or_move_in_df.apply(lambda row: exit_or_move_in(row), axis=1)
        total_exit_to_or_move_in_df = pd.DataFrame(total_exit_to_or_move_in_df.groupby(['static_demographics.race_text','exit_dest_or_move_in'])['clients.unique_identifier'].nunique()).reset_index()
        total_exit_to_or_move_in_df['Permanent Destination'] = total_exit_to_or_move_in_df.apply(lambda row: ph_category1(row), axis=1)
        total_exit_to_or_move_in_df = pd.DataFrame(total_exit_to_or_move_in_df.groupby(['static_demographics.race_text', 'Permanent Destination'])['clients.unique_identifier'].sum()).reset_index()
        total_exit_to_or_move_in_df['Permanent Destination % by Race'] = total_exit_to_or_move_in_df['clients.unique_identifier']/total_exit_to_or_move_in_df['clients.unique_identifier'].sum()

        total_exit_to_or_move_in_df['Reporting Month'] = REPORT_START_DATE.strftime("%b-%Y")
        total_exit_to_or_move_in_df['Reporting Window'] = time_range

        list_of_dfs_4.append(total_exit_to_or_move_in_df)
        
    REPORT_START_DATE = REPORT_START_DATE + relativedelta(months=1)

In [21]:
df4 = pd.concat(list_of_dfs_4)
df4.to_csv(f'{PATH}/newsletter_housed_counts_by_destination_by_race.csv')

# FTH Racial Equity Data 

In [22]:
def binary_race_variable(row, race):
    if row['static_demographics.race_text']==race:
        return race
    else:
        return f'Non-{race}'

fth_binary_race_data_d = {}
df1 = df.copy()
# df1['Binary Race Variable'] = df1.apply(lambda row: binary_race_variable(row, race), axis=1)

rep_period_start = pd.to_datetime('2020-01-01')
rep_period_end = rep_period_start + relativedelta(months=12)
while rep_period_start<pd.to_datetime('2023-01-01'):
    temp = df1.copy()
    proj_type_list = temp['programs.project_type_code'].unique()
    race_list = temp['static_demographics.race_text'].unique()
    fth_binary_race_data_d[rep_period_start.strftime('%Y')] = {}
    for race in race_list:
        if race in ['White', 'Black, African American, or African', 'Native Hawaiian or Pacific Islander', 'Multi-Racial', 'American Indian, Alaska Native, or Indigenous', 'Asian or Asian American']:
            temp = df1.copy()
            temp['Binary Race Variable'] = temp.apply(lambda row: binary_race_variable(row, race), axis=1)
            temp = pd.DataFrame(getFirstTimeHomelessDF(temp, rep_period_start, rep_period_end).groupby(['Binary Race Variable','programs.project_type_code'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "FTH Count"}).reset_index()
            temp = pd.DataFrame(temp.pivot(index='Binary Race Variable', columns='programs.project_type_code', values='FTH Count')).reset_index()
            temp[[x for x in temp.columns if x in proj_type_list]] = temp[[x for x in temp.columns if x in proj_type_list]].fillna(0).astype(int)
            temp['Year'] = rep_period_start.strftime('%Y')
            fth_binary_race_data_d[rep_period_start.strftime('%Y')][race] = temp.copy()

    # Set repording period start and end
    rep_period_start = rep_period_end
    rep_period_end = rep_period_start + relativedelta(months=12)

In [23]:
fth_binary_race_data_df = pd.DataFrame()
for key in fth_binary_race_data_d:
    for k in fth_binary_race_data_d[key]:
        if fth_binary_race_data_df.empty:
            fth_binary_race_data_df = fth_binary_race_data_d[key][k]
        else:
            fth_binary_race_data_df = pd.concat([fth_binary_race_data_df, fth_binary_race_data_d[key][k]])

In [24]:
fth_binary_race_data_df[fth_binary_race_data_df.columns[1:]] = fth_binary_race_data_df[fth_binary_race_data_df.columns[1:]].fillna(0).astype(int)
fth_binary_race_data_df = fth_binary_race_data_df.reset_index(drop=True)

In [25]:
fth_binary_race_data_df.to_csv(f'{PATH}/binary_race_data_counts_df.csv')

In [26]:
fth_d = {}

population_percent = {
    "White":.746,
    "Black, African American, or African":.056,
    'American Indian, Alaska Native, or Indigenous':.014,
    'Asian or Asian American':.129,
    'Native Hawaiian or Pacific Islander':0.006,
    'Multi-Racial':.049
    }
for year in fth_binary_race_data_d:
    fth_d[year] = {}
    for race in fth_binary_race_data_d[year]:
        if race in population_percent: 
            fth_d[year][race] = {}
            for col in [x for x in fth_binary_race_data_d[year][race].columns if x in proj_type_list]:
                fth_d[year][race][col] = {}
                for var in fth_binary_race_data_d[year][race]['Binary Race Variable'].unique():
                    if var==race:
                        fth_d[year][race][col][var] = (fth_binary_race_data_d[year][race][fth_binary_race_data_d[year][race]['Binary Race Variable']==race][col].values[0]/(population_percent[race]*3298634))*10000
                    else:
                        fth_d[year][race][col][var] = (fth_binary_race_data_d[year][race][fth_binary_race_data_d[year][race]['Binary Race Variable']!=race][col].values[0]/((1-population_percent[race])*3298634))*10000

In [27]:
fth_race_data_df = pd.DataFrame()
for year in fth_d:
    for race in fth_d[year]:
        temp_df = pd.DataFrame(fth_d[year][race])
        temp_df['Year'] = year
        if fth_race_data_df.empty:
            fth_race_data_df = temp_df.copy()
        else:
            fth_race_data_df = pd.concat([fth_race_data_df, temp_df])
            
fth_race_data_df = fth_race_data_df.reset_index()
fth_race_data_df = fth_race_data_df.rename(columns={'index':'Binary Race Variable'})

In [28]:
fth_race_data_df.to_csv(f'{PATH}/binary_race_data_percent_by_pop_df.csv')

In [29]:
fth_likely_d = {}
for year in fth_d:
    fth_likely_d[year] = {}
    for race in fth_d[year]:
        fth_likely_d[year][race] = {}
        for key in fth_d[year][race]:
            if fth_d[year][race][key][f'Non-{race}']!=0:
                fth_likely_d[year][race][key] = fth_d[year][race][key][race]/fth_d[year][race][key][f'Non-{race}']
            else:
                fth_likely_d[year][race][key] = None

In [30]:
fth_likely_data_df = pd.DataFrame()
for year in fth_likely_d:
    temp_df = pd.DataFrame(fth_likely_d[year]).T
    temp_df['Year'] = year
    if fth_likely_data_df.empty:
        fth_likely_data_df = temp_df.copy()
    else:
        fth_likely_data_df = pd.concat([fth_likely_data_df, temp_df])

fth_likely_data_df = fth_likely_data_df.reset_index().rename(columns={'index':'Binary Race Variable'})
fth_likely_data_df[fth_likely_data_df.columns[1:]] = fth_likely_data_df[fth_likely_data_df.columns[1:]]

In [31]:
fth_likely_data_df.to_csv(f'{PATH}/binary_race_likeliness_data_df.csv')

# Active Racial Equity Data

In [32]:
def binary_race_variable(row, race):
    if row['static_demographics.race_text']==race:
        return race
    else:
        return f'Non-{race}'

active_binary_race_data_d = {}
df1 = df.copy()
# df1['Binary Race Variable'] = df1.apply(lambda row: binary_race_variable(row, race), axis=1)

rep_period_start = pd.to_datetime('2020-01-01')
rep_period_end = rep_period_start + relativedelta(months=12)
while rep_period_start<pd.to_datetime('2023-01-01'):
    temp = df1.copy()
    proj_type_list = temp['programs.project_type_code'].unique()
    race_list = temp['static_demographics.race_text'].unique()
    active_binary_race_data_d[rep_period_start.strftime('%Y')] = {}
    for race in race_list:
        if race in ['White', 'Black, African American, or African', 'Native Hawaiian or Pacific Islander', 'Multi-Racial', 'American Indian, Alaska Native, or Indigenous', 'Asian or Asian American']:
            temp = df1.copy()
            temp['Binary Race Variable'] = temp.apply(lambda row: binary_race_variable(row, race), axis=1)
            temp = pd.DataFrame(getActiveClientsDF(temp, rep_period_start, rep_period_end).groupby(['Binary Race Variable','programs.project_type_code'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "Active Count"}).reset_index()
            temp = pd.DataFrame(temp.pivot(index='Binary Race Variable', columns='programs.project_type_code', values='Active Count')).reset_index()
            temp[[x for x in temp.columns if x in proj_type_list]] = temp[[x for x in temp.columns if x in proj_type_list]].fillna(0).astype(int)
            temp['Year'] = rep_period_start.strftime('%Y')
            active_binary_race_data_d[rep_period_start.strftime('%Y')][race] = temp.copy()

    # Set repording period start and end
    rep_period_start = rep_period_end
    rep_period_end = rep_period_start + relativedelta(months=12)

In [33]:
active_binary_race_data_df = pd.DataFrame()
for key in active_binary_race_data_d:
    for k in active_binary_race_data_d[key]:
        if active_binary_race_data_df.empty:
            active_binary_race_data_df = active_binary_race_data_d[key][k]
        else:
            active_binary_race_data_df = pd.concat([active_binary_race_data_df, active_binary_race_data_d[key][k]])

In [34]:
active_binary_race_data_df[active_binary_race_data_df.columns[1:]] = active_binary_race_data_df[active_binary_race_data_df.columns[1:]].fillna(0).astype(int)
active_binary_race_data_df = active_binary_race_data_df.reset_index(drop=True)

In [35]:
active_binary_race_data_df.to_csv(f'{PATH}/active_binary_race_data_counts_df.csv')

In [36]:
active_d = {}

population_percent = {
    "White":.746,
    "Black, African American, or African":.056,
    'American Indian, Alaska Native, or Indigenous':.014,
    'Asian or Asian American':.129,
    'Native Hawaiian or Pacific Islander':0.006,
    'Multi-Racial':.049
    }
for year in active_binary_race_data_d:
    active_d[year] = {}
    for race in active_binary_race_data_d[year]:
        if race in population_percent: 
            active_d[year][race] = {}
            for col in [x for x in active_binary_race_data_d[year][race].columns if x in proj_type_list]:
                active_d[year][race][col] = {}
                for var in active_binary_race_data_d[year][race]['Binary Race Variable'].unique():
                    if var==race:
                        active_d[year][race][col][var] = (active_binary_race_data_d[year][race][active_binary_race_data_d[year][race]['Binary Race Variable']==race][col].values[0]/(population_percent[race]*3298634))*10000
                    else:
                        active_d[year][race][col][var] = (active_binary_race_data_d[year][race][active_binary_race_data_d[year][race]['Binary Race Variable']!=race][col].values[0]/((1-population_percent[race])*3298634))*10000

In [37]:
active_race_data_df = pd.DataFrame()
for year in active_d:
    for race in active_d[year]:
        temp_df = pd.DataFrame(active_d[year][race])
        temp_df['Year'] = year
        if active_race_data_df.empty:
            active_race_data_df = temp_df.copy()
        else:
            active_race_data_df = pd.concat([active_race_data_df, temp_df])
            
active_race_data_df = active_race_data_df.reset_index()
active_race_data_df = active_race_data_df.rename(columns={'index':'Binary Race Variable'})

In [38]:
active_race_data_df.to_csv(f'{PATH}/active_binary_race_data_percent_by_pop_df.csv')

In [39]:
active_likely_d = {}
for year in active_d:
    active_likely_d[year] = {}
    for race in active_d[year]:
        active_likely_d[year][race] = {}
        for key in active_d[year][race]:
            if active_d[year][race][key][f'Non-{race}']!=0:
                active_likely_d[year][race][key] = active_d[year][race][key][race]/active_d[year][race][key][f'Non-{race}']
            else:
                active_likely_d[year][race][key] = None

In [40]:
active_likely_data_df = pd.DataFrame()
for year in active_likely_d:
    temp_df = pd.DataFrame(active_likely_d[year]).T
    temp_df['Year'] = year
    if active_likely_data_df.empty:
        active_likely_data_df = temp_df.copy()
    else:
        active_likely_data_df = pd.concat([active_likely_data_df, temp_df])

active_likely_data_df = active_likely_data_df.reset_index().rename(columns={'index':'Binary Race Variable'})
active_likely_data_df[active_likely_data_df.columns[1:]] = active_likely_data_df[active_likely_data_df.columns[1:]]

In [41]:
active_likely_data_df.to_csv(f'{PATH}/active_binary_race_likeliness_data_df.csv')

# Housed Racial Equity Data

In [42]:
def binary_race_variable(row, race):
    if row['static_demographics.race_text']==race:
        return race
    else:
        return f'Non-{race}'

housed_binary_race_data_d = {}
df1 = df.copy()
# df1['Binary Race Variable'] = df1.apply(lambda row: binary_race_variable(row, race), axis=1)

rep_period_start = pd.to_datetime('2020-01-01')
rep_period_end = rep_period_start + relativedelta(months=12)
while rep_period_start<pd.to_datetime('2023-01-01'):
    temp = df1.copy()
    proj_type_list = temp['programs.project_type_code'].unique()
    race_list = temp['static_demographics.race_text'].unique()
    housed_binary_race_data_d[rep_period_start.strftime('%Y')] = {}
    for race in race_list:
        if race in ['White', 'Black, African American, or African', 'Native Hawaiian or Pacific Islander', 'Multi-Racial', 'American Indian, Alaska Native, or Indigenous', 'Asian or Asian American']:
            temp = df1.copy()
            temp['Binary Race Variable'] = temp.apply(lambda row: binary_race_variable(row, race), axis=1)
            temp = pd.DataFrame(getClientsExitedAndHoused(temp, rep_period_start, rep_period_end).groupby(['Binary Race Variable','programs.project_type_code'])['clients.unique_identifier'].nunique().sort_values(ascending=False)).rename(columns={"clients.unique_identifier": "housed Count"}).reset_index()
            temp = pd.DataFrame(temp.pivot(index='Binary Race Variable', columns='programs.project_type_code', values='housed Count')).reset_index()
            temp[[x for x in temp.columns if x in proj_type_list]] = temp[[x for x in temp.columns if x in proj_type_list]].fillna(0).astype(int)
            temp['Year'] = rep_period_start.strftime('%Y')
            housed_binary_race_data_d[rep_period_start.strftime('%Y')][race] = temp.copy()

    # Set repording period start and end
    rep_period_start = rep_period_end
    rep_period_end = rep_period_start + relativedelta(months=12)

In [43]:
housed_binary_race_data_df = pd.DataFrame()
for key in housed_binary_race_data_d:
    for k in housed_binary_race_data_d[key]:
        if housed_binary_race_data_df.empty:
            housed_binary_race_data_df = housed_binary_race_data_d[key][k]
        else:
            housed_binary_race_data_df = pd.concat([housed_binary_race_data_df, housed_binary_race_data_d[key][k]])

In [44]:
housed_binary_race_data_df[housed_binary_race_data_df.columns[1:]] = housed_binary_race_data_df[housed_binary_race_data_df.columns[1:]].fillna(0).astype(int)
housed_binary_race_data_df = housed_binary_race_data_df.reset_index(drop=True)

In [45]:
housed_binary_race_data_df.to_csv(f'{PATH}/housed_binary_race_data_counts_df.csv')

In [46]:
housed_d = {}

population_percent = {
    "White":.746,
    "Black, African American, or African":.056,
    'American Indian, Alaska Native, or Indigenous':.014,
    'Asian or Asian American':.129,
    'Native Hawaiian or Pacific Islander':0.006,
    'Multi-Racial':.049
    }
for year in housed_binary_race_data_d:
    housed_d[year] = {}
    for race in housed_binary_race_data_d[year]:
        if race in population_percent: 
            housed_d[year][race] = {}
            for col in [x for x in housed_binary_race_data_d[year][race].columns if x in proj_type_list]:
                housed_d[year][race][col] = {}
                for var in housed_binary_race_data_d[year][race]['Binary Race Variable'].unique():
                    if var==race:
                        housed_d[year][race][col][var] = (housed_binary_race_data_d[year][race][housed_binary_race_data_d[year][race]['Binary Race Variable']==race][col].values[0]/(population_percent[race]*3298634))*10000
                    else:
                        housed_d[year][race][col][var] = (housed_binary_race_data_d[year][race][housed_binary_race_data_d[year][race]['Binary Race Variable']!=race][col].values[0]/((1-population_percent[race])*3298634))*10000

In [47]:
housed_race_data_df = pd.DataFrame()
for year in housed_d:
    for race in housed_d[year]:
        temp_df = pd.DataFrame(housed_d[year][race])
        temp_df['Year'] = year
        if housed_race_data_df.empty:
            housed_race_data_df = temp_df.copy()
        else:
            housed_race_data_df = pd.concat([housed_race_data_df, temp_df])
            
housed_race_data_df = housed_race_data_df.reset_index()
housed_race_data_df = housed_race_data_df.rename(columns={'index':'Binary Race Variable'})

In [48]:
housed_race_data_df.to_csv(f'{PATH}/housed_binary_race_data_percent_by_pop_df.csv')

In [49]:
housed_likely_d = {}
for year in housed_d:
    housed_likely_d[year] = {}
    for race in housed_d[year]:
        housed_likely_d[year][race] = {}
        for key in housed_d[year][race]:
            if housed_d[year][race][key][f'Non-{race}']!=0:
                housed_likely_d[year][race][key] = housed_d[year][race][key][race]/housed_d[year][race][key][f'Non-{race}']
            else:
                housed_likely_d[year][race][key] = None

In [50]:
housed_likely_data_df = pd.DataFrame()
for year in housed_likely_d:
    temp_df = pd.DataFrame(housed_likely_d[year]).T
    temp_df['Year'] = year
    if housed_likely_data_df.empty:
        housed_likely_data_df = temp_df.copy()
    else:
        housed_likely_data_df = pd.concat([housed_likely_data_df, temp_df])

housed_likely_data_df = housed_likely_data_df.reset_index().rename(columns={'index':'Binary Race Variable'})
housed_likely_data_df[housed_likely_data_df.columns[1:]] = housed_likely_data_df[housed_likely_data_df.columns[1:]]

In [51]:
housed_likely_data_df.to_csv(f'{PATH}/housed_binary_race_likeliness_data_df.csv')

# PIT DATA

In [52]:
xls = pd.ExcelFile('assets/PIT Data.xlsx')
pit_data_d = {}

for sheet in xls.sheet_names:
    pit_data_d[sheet.split(' ')[0]] = pd.read_excel(xls, sheet)
    pit_data_d[sheet.split(' ')[0]][pit_data_d[sheet.split(' ')[0]].columns[1:]] = pit_data_d[sheet.split(' ')[0]][pit_data_d[sheet.split(' ')[0]].columns[1:]].fillna(0).astype(int)
    pit_data_d[sheet.split(' ')[0]]['Sheltered'] = pit_data_d[sheet.split(' ')[0]]['Emergency'] + pit_data_d[sheet.split(' ')[0]]['Transitional'] + pit_data_d[sheet.split(' ')[0]]['Safe Haven']
    pit_data_d[sheet.split(' ')[0]] = pit_data_d[sheet.split(' ')[0]][['Unnamed: 0', 'Sheltered', 'Unsheltered', 'Total']]
    pit_data_d[sheet.split(' ')[0]] = pit_data_d[sheet.split(' ')[0]].T
    pit_data_d[sheet.split(' ')[0]].columns = pit_data_d[sheet.split(' ')[0]].iloc[0, :]
    pit_data_d[sheet.split(' ')[0]] = pit_data_d[sheet.split(' ')[0]][1:]
    pit_data_d[sheet.split(' ')[0]].columns = [x.replace('\n', '') for x in pit_data_d[sheet.split(' ')[0]].columns]
    pit_data_d[sheet.split(' ')[0]] = pit_data_d[sheet.split(' ')[0]].iloc[:,-6:]
    if sheet.split(' ')[0] in ['2020', '2021']:
        pit_data_d[sheet.split(' ')[0]] = pit_data_d[sheet.split(' ')[0]].rename(columns=
                                                                                 {'American Indian or Alaska Native':'American Indian, Alaska Native, or Indigenous',
                                                                                  'Asian':'Asian or Asian American',
                                                                                  'Black or African-American':'Black, African American, or African',
                                                                                  'Native Hawaiian or Other Pacific Islander':'Native Hawaiian or Pacific Islander',
                                                                                  })
    pit_data_d[sheet.split(' ')[0]] = pit_data_d[sheet.split(' ')[0]].T.reset_index().rename(columns={'index':'static_demographics.race_text'})
    pit_data_d[sheet.split(' ')[0]]['Year'] = int(sheet.split(' ')[0])

pit_data_df = pd.DataFrame()
for key in pit_data_d:
    if pit_data_df.empty:
        pit_data_df = pit_data_d[key]
    else:
        pit_data_df = pd.concat([pit_data_df, pit_data_d[key]])
pit_data_df = pit_data_df.reset_index(drop=True)

In [53]:
binary_pit_data_d = {}
for key in pit_data_d:
    binary_pit_data_d[key] = {}
    for race in pit_data_d[key]['static_demographics.race_text'].unique():
        temp = pit_data_d[key].copy()
        temp['Binary Race Variable'] = temp.apply(lambda row: binary_race_variable(row, race), axis=1)
        temp = pd.merge(pd.DataFrame(temp.groupby('Binary Race Variable')['Sheltered'].sum()).reset_index(), pd.DataFrame(temp.groupby('Binary Race Variable')['Unsheltered'].sum()).reset_index(), on='Binary Race Variable', how='left')
        temp['Year'] = int(key)
        binary_pit_data_d[key][race] = temp.copy()

In [54]:
binary_pit_data_df = pd.DataFrame()

for year in binary_pit_data_d:
    for race in binary_pit_data_d[year]:
        if binary_pit_data_df.empty:
            binary_pit_data_df = binary_pit_data_d[year][race].copy()
        else:
            binary_pit_data_df = pd.concat([binary_pit_data_df, binary_pit_data_d[year][race].copy()])

In [55]:
binary_pit_data_df.to_csv(f'{PATH}/binary_pit_data_df.csv')

In [56]:
calc_pit_data_d = {}

population_percent = {
    "White":.746,
    "Black, African American, or African":.056,
    'American Indian, Alaska Native, or Indigenous':.014,
    'Asian or Asian American':.129,
    'Native Hawaiian or Pacific Islander':0.006,
    'Multi-Racial':.049
    }
for year in binary_pit_data_d:
    calc_pit_data_d[year] = {}
    for race in binary_pit_data_d[year]:
        if race in population_percent: 
            calc_pit_data_d[year][race] = {}
            for col in [x for x in binary_pit_data_d[year][race].columns if x in ['Sheltered', 'Unsheltered']]:
                calc_pit_data_d[year][race][col] = {}
                for var in binary_pit_data_d[year][race]['Binary Race Variable'].unique():
                    if var==race:
                        calc_pit_data_d[year][race][col][var] = (binary_pit_data_d[year][race][binary_pit_data_d[year][race]['Binary Race Variable']==race][col].values[0]/(population_percent[race]*3298634))*10000
                    else:
                        calc_pit_data_d[year][race][col][var] = (binary_pit_data_d[year][race][binary_pit_data_d[year][race]['Binary Race Variable']!=race][col].values[0]/((1-population_percent[race])*3298634))*10000

In [57]:
calc_pit_data_df = pd.DataFrame()

for year in calc_pit_data_d:
    for race in calc_pit_data_d[year]:
        temp = pd.DataFrame(calc_pit_data_d[year][race])
        temp['Year'] = int(year)
        if calc_pit_data_df.empty:
            calc_pit_data_df = temp.copy()
        else:
            calc_pit_data_df = pd.concat([calc_pit_data_df, temp.copy()])

In [58]:
calc_pit_data_df.to_csv(f'{PATH}/calc_pit_data_df.csv')

In [59]:
pit_data_likely_d = {}
for year in calc_pit_data_d:
    pit_data_likely_d[year] = {}
    for race in calc_pit_data_d[year]:
        pit_data_likely_d[year][race] = {}
        for key in calc_pit_data_d[year][race]:
            if calc_pit_data_d[year][race][key][f'Non-{race}']!=0:
                pit_data_likely_d[year][race][key] = calc_pit_data_d[year][race][key][race]/calc_pit_data_d[year][race][key][f'Non-{race}']
            else:
                pit_data_likely_d[year][race][key] = None

In [60]:
pit_data_likely_df = pd.DataFrame()


for year in pit_data_likely_d:
    for race in pit_data_likely_d[year]:
        temp = pd.DataFrame(pd.Series(pit_data_likely_d[year][race]))
        temp = temp.rename(columns={0:'Likely Metric'})
        temp['Year'] = int(year)
        temp['Race'] = race
        if pit_data_likely_df.empty:
            pit_data_likely_df = temp.copy()
        else:
            pit_data_likely_df = pd.concat([pit_data_likely_df, temp.copy()])
pit_data_likely_df = pit_data_likely_df.reset_index()

In [61]:
pit_data_likely_df.to_csv(f'{PATH}/pit_data_likely_df.csv')

In [62]:
pit_data_df.to_csv(f'{PATH}/pit_data_df.csv')